[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aurelio-labs/langchain-course/blob/main/chapters/05-agents-intro.ipynb)

#### LangChain Essentials Course

# LangChain Agents Intro

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Agents, adding the ability to use tools such as search and calculators to complete tasks that normal LLMs cannot fufil. In this example we will be using Gemini's `flash-2.5-lite`.

In [1]:
!pip uninstall -y langchain google-generativeai
!pip install -qU \
  "pydantic>=2.11.4,<3" \
  "langchain-core>=0.3.70,<0.4" \
  "langchain-community>=0.3.27,<0.4" \
  "langchain-google-genai>=2.1.10" \
  "google-genai>=0.7.0" \
  "langsmith>=0.3.4" \
  google-search-results


Found existing installation: langchain 0.3.27
Uninstalling langchain-0.3.27:
  Successfully uninstalled langchain-0.3.27
Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.7/241.7 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.5/378.5 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5

---

> ⚠️ If using LangSmith, add your API key below:

In [2]:
import os
from getpass import getpass
from google.colab import userdata

# must enter API key
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY") or \
    getpass("Enter LangSmith API Key: ")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "aurelioai-langchain-course-agents-intro-openai"

---

## Introduction to Tools

Tools are a way augment our LLMs with code execution. A tool is simply a function formatted so that our agent can undertstand how to use it, and then execute it. Let's start by creating a few simple tools.

We can use the `@tool` decorator to create an LLM-compatible tool from a standard python function — this function should include a few things for optimal performance:

* A docstring describing what the tool does and when it should be used, this will be read by our LLM/agent and used to decide when to use the tool, and also how to use the tool.

* Clear parameter names that ideally tell the LLM what each parameter is, if it isn't clear we make sure the docstring explains what the parameter is for and how to use it.

* Both parameter and return type annotations.

In [3]:
from langchain_core.tools import tool

@tool
def add(x: float, y: float) -> float:
    """Add 'x' and 'y'."""
    return x + y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply 'x' and 'y'."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Raise 'x' to the power of 'y'."""
    return x ** y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract 'x' from 'y'."""
    return y - x

With the `@tool` decorator our function is turned into a `StructuredTool` object, which we can see below:

In [4]:
add

StructuredTool(name='add', description="Add 'x' and 'y'.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x7faba25aed40>)

We can see the tool name, description, and arg schema:

In [5]:
print(f"{add.name=}\n{add.description=}")

add.name='add'
add.description="Add 'x' and 'y'."


In [6]:
add.args_schema.model_json_schema()

{'description': "Add 'x' and 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [7]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'x' to the power of 'y'.",
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'exponentiate',
 'type': 'object'}

When invoking the tool, a JSON string output by the LLM will be parsed into JSON and then consumed as kwargs, similar to the below:

In [8]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

This is then passed into the tool function as `kwargs` (keyword arguments) as indicated by the `**` operator - the `**` operator is used to unpack the dictionary into keyword arguments.

In [9]:
exponentiate.func(**llm_output_dict)

25

This covers the basics of tools and how they work, let's move on to creating the agent itself.

## Creating an Agent

We're going to construct a simple tool calling agent. We will use **L**ang**C**hain **E**pression **L**anguage (LCEL) to construct the agent. We will cover LCEL more in the next chapter, but for now - all we need to know is that our agent will be constructed using syntax and components like so:


```
agent = (
    <input parameters, including chat history and user query>
    | <prompt>
    | <LLM with tools>
)
```

We need this agent to remember previous interactions within the conversation. To do that, we will use the `ChatPromptTemplate` with a system message, a placeholder for our chat history, a placeholder for the user query, and finally a placeholder for the agent scratchpad.

The agent scratchpad is where the agent will write it's _"notes"_ as it is working through multiple internal thought and tool-use steps to produce a final output to the user.

In [10]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

Next, we must define our LLM, we will use the `gemini` model with a `temperature` of `0.0`.

In [11]:
from langchain_google_genai import ChatGoogleGenerativeAI

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY") or getpass(
    "Enter Gemini API Key: "
)

openai_model = "gemini-2.5-flash-lite"

llm = ChatGoogleGenerativeAI(
    model=openai_model,
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)


When creating an agent we need to add conversational memory to make the agent remember previous interactions. We'll be using the older `ConversationBufferMemory` class rather than the newer `RunnableWithMessageHistory` — the reason being that we will also be using the older `create_tool_calling_agent` and `AgentExecutor` method and class.

In the `05` chapter we will be using the newer `RunnableWithMessageHistory` class as we'll be building a custom `AgentExecutor`.

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

/tmp/ipython-input-287820297.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


Now we will initialize our agent. For that we need:

* `llm`: as already defined
* `tools`: to be defined (just a list of our previously defined tools)
* `prompt`: as already defined
* `memory`: as already defined

In [13]:
from langchain.agents import create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

Our `agent` by itself is like one-step of our agent execution loop. So, if we call the `agent.invoke` method it will get the LLM to generate a single response and go no further, so no tools will be executed, and no next iterations will be performed.

We can see this by asking a query that should trigger a tool call:

In [14]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'y': 7.68, 'x': 10.7}, log="\nInvoking: `multiply` with `{'y': 7.68, 'x': 10.7}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"y": 7.68, "x": 10.7}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--e3f49a9c-1068-4fef-813d-0a2eb772d6b9-0', tool_calls=[{'name': 'multiply', 'args': {'y': 7.68, 'x': 10.7}, 'id': 'cf9ae9ce-5581-4dff-8cb4-8deb8fa2dcda', 'type': 'tool_call'}], usage_metadata={'input_tokens': 222, 'output_tokens': 24, 'total_tokens': 246, 'input_token_details': {'cache_read': 0}})], tool_call_id='cf9ae9ce-5581-4dff-8cb4-8deb8fa2dcda')]

Here, we can see the LLM has generated that we should use the `multiply` tool and the tool input should be `{"x": 10.7, "y": 7.68}`. However, the tool is not executed. For that to happen we need an agent execution loop, which will handle the multiple iterations of generation to tool calling to generation, etc.

We use the `AgentExecutor` class to handle the execution loop:

In [15]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

Now let's try the same query with the executor, note that the `intermediate_steps` parameter that we added before is no longer needed as the executor handles it internally.

In [16]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'y': 7.68, 'x': 10.7}`


82.17599999999999

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={})],
 'output': ''}

We can see that the `multiply` tool was invoked, producing the observation of `82.175999...`. After the observation was provided, we can see that the LLM then generated a final response of:

```
10.7 multiplied by 7.68 is approximately 82.18.
```

This final response was generated based on the original query and the tool output (ie the _observation_). We can also confirm that this answer is accurate:

In [17]:
10.7*7.68

82.17599999999999

Let's test our agent with some memory and tool use. First, we tell it our name, then we will perform a few tool calls, then see if the agent can still recall our name.

First, give the agent our name:

In [18]:
agent_executor.invoke({
    "input": "My name is James",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'y': 7.68, 'x': 10.7}`


82.17599999999999

> Finished chain.


{'input': 'My name is James',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={})],
 'output': ''}

Now let's try and get the agent to perform multiple tool calls within a single execution loop:

In [20]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'y': 10.0, 'x': 9.0}`


19.0
Invoking: `subtract` with `{'y': 8.0, 'x': 19.0}`


-11.0
Invoking: `exponentiate` with `{'y': 3.0, 'x': 11.0}`


1331.0
Invoking: `add` with `{'y': 10.0, 'x': 9.0}`


19.0
Invoking: `multiply` with `{'y': 2.0, 'x': 4.0}`


8.0
Invoking: `exponentiate` with `{'y': 3.0, 'x': 8.0}`


512.0
Invoking: `subtract` with `{'y': 512.0, 'x': 19.0}`


493.0The answer to your question is 493.

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello James, 10.7 multiplied by 7.68 is 82.176.\n\nThe answer to your question is 493.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer to your question is 493.', additional_kwargs={}, response_metadata={})],
 'output': 'The answer to your question is 493.'}

Let's confirm that the answer is accurate:

In [21]:
9+10-(4*2)**3

-493

Perfect, now let's see if the agent can still recall our name:

In [22]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Your name is James.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is James', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello James, 10.7 multiplied by 7.68 is 82.176.\n\nThe answer to your question is 493.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The answer to your question is 493.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name 

The agent has successfully recalled our name. Let's move on to another agent example.

## SerpAPI Weather Agent

In this example, we'll be using the same agent and executor setup as before, but we'll be adding the [SerpAPI](https://serpapi.com/users/sign_in) service to allow our agent to search the web for information.

To use this tool, you need an API key, with the free plan you can use up to 100 searches per month.

In [23]:
os.environ["SERPAPI_API_KEY"] = userdata.get("SERPAPI_API_KEY") \
    or getpass("Enter your SerpAPI API key: ")

Here we will load the `serpapi` tool directly from the prebuilt tools that LangChain provides.

In [24]:
from langchain.agents import load_tools

toolbox = load_tools(tool_names=['serpapi'], llm=llm)

These custom tools can look into your IP address, find out where you are currently, then we will also use a secondary function to get the current date and time, then we will use this information to feed into the SerpAPI to find us the weather pattern in your area and at the time of the function calling.

In [25]:
import requests
from datetime import datetime

@tool
def get_location_from_ip():
    """Get the geographical location based on the IP address."""
    try:
        response = requests.get("https://ipinfo.io/json")
        data = response.json()
        if 'loc' in data:
            latitude, longitude = data['loc'].split(',')
            data = (
                f"Latitude: {latitude},\n"
                f"Longitude: {longitude},\n"
                f"City: {data.get('city', 'N/A')},\n"
                f"Country: {data.get('country', 'N/A')}"
            )
            return data
        else:
            return "Location could not be determined."
    except Exception as e:
        return f"Error occurred: {e}"

@tool
def get_current_datetime() -> str:
    """Return the current date and time."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

We can create our prompt, this time we'll skip the `chat_history` part as we don't need it. However, you can add it if preferred.

In [33]:
from langchain.tools.render import render_text_description
tools = toolbox + [get_current_datetime, get_location_from_ip]
tool_descriptions = render_text_description(tools)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful weather assistant. You can use tools.\n\n"
     "TOOLS AVAILABLE:\n"
     f"{tool_descriptions}\n\n"
     "When you need current info (date/time, location, weather), use the tools.\n"
     "Reason step-by-step. If a tool fails, try another strategy or refine your query."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])

Now we create our full `tools` list, our `agent`, and the `agent_executor`:

In [34]:
agent = create_tool_calling_agent(
    llm=llm, tools=tools, prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True
)

For me I have to specify to the AI as I live in the UK and alot of places in the UK also exist in USA, which is why I explicitly state to use the country in the search as well as the town / city.

In [37]:
out = agent_executor.invoke({
    "input": (
        "I have a few questions, what is the date and time right now? "
        "How is the weather where I am? Please give me degrees in Celsius. Use my ip location if needed"
    )
})



> Entering new AgentExecutor chain...

Invoking: `get_current_datetime` with `{}`


2025-09-02 18:26:14
Invoking: `get_location_from_ip` with `{}`


Latitude: 36.1750,
Longitude: -115.1372,
City: Las Vegas,
Country: US
Invoking: `Search` with `{'query': 'weather in Las Vegas Celsius'}`


{'type': 'weather_result', 'temperature': '34', 'unit': 'Celsius', 'precipitation': '0%', 'humidity': '23%', 'wind': '2 km/h', 'location': 'Las Vegas, NV', 'date': 'Tuesday 11:00 AM', 'weather': 'Cloudy'}The current date and time is September 2, 2025, 6:26:14 PM. The weather in Las Vegas, NV is cloudy with a temperature of 34°C, 23% humidity, and 2 km/h wind.

> Finished chain.


In [38]:
from IPython.display import display, Markdown

display(Markdown(out["output"]))

The current date and time is September 2, 2025, 6:26:14 PM. The weather in Las Vegas, NV is cloudy with a temperature of 34°C, 23% humidity, and 2 km/h wind.

That's the correct answer, and we even get the approximate answer in Celsius despite the tool returning the temperature in Fahrenheit.

We've finished our into to LangChain Agents, in the next chapter we will be looking at how to create custom agents and executors.

---

'Search(query: str, **kwargs: Any) -> str - A search engine. Useful for when you need to answer questions about current events. Input should be a search query.\nget_current_datetime() -> str - Return the current date and time.\nget_location_from_ip() - Get the geographical location based on the IP address.'